# Production Technology

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

In [8]:
import pandas as pd 
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS, RandomEffects
from scipy.stats import chi2

In [9]:
# Indlæs data fra firms.csv
dat = pd.read_csv('firms.csv')

# Konverter til paneldata format (hvorfor paneldata? fordi vi skal estimere en lineær paneldata-model for Cobb-Douglas produktionsfunktionen.)
dat['year'] = dat.index % 12  
dat = dat.set_index(['year', dat.index])

# Estimer Pooled OLS
pooled_ols = smf.ols('ldsa ~ lcap + lemp', data=dat).fit()

# Estimer Fixed Effects (FE)
fe_model = PanelOLS.from_formula('ldsa ~ lcap + lemp + EntityEffects', data=dat).fit()

# Estimer Random Effects (RE)
re_model = RandomEffects.from_formula('ldsa ~ lcap + lemp', data=dat).fit()

# Hypotesetest: Konstant skalaafkast
beta_k, beta_l = fe_model.params['lcap'], fe_model.params['lemp']
wald_stat = ((beta_k + beta_l - 1)**2) / (fe_model.cov.iloc[0, 0] + fe_model.cov.iloc[1, 1])
p_value = 1 - chi2.cdf(wald_stat, 1)

# Hausman-test mellem FE og RE
diff = fe_model.params - re_model.params
var_diff = fe_model.cov + re_model.cov
hausman_stat = diff.T @ np.linalg.inv(var_diff) @ diff
hausman_p_value = 1 - chi2.cdf(hausman_stat, len(diff))

# Udskriv resultater
print("Pooled OLS:\n", pooled_ols.summary())
print("\nFixed Effects:\n", fe_model.summary)
print("\nRandom Effects:\n", re_model.summary)
print(f"\nWald-test for konstant skalaafkast: Test-statistik = {wald_stat:.3f}, p-værdi = {p_value:.3f}")
print(f"\nHausman-test mellem FE og RE: Test-statistik = {hausman_stat:.3f}, p-værdi = {hausman_p_value:.3f}")

Pooled OLS:
                             OLS Regression Results                            
Dep. Variable:                   ldsa   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                 2.807e+04
Date:                Tue, 18 Feb 2025   Prob (F-statistic):               0.00
Time:                        11:35:57   Log-Likelihood:                -2125.9
No. Observations:                5292   AIC:                             4258.
Df Residuals:                    5289   BIC:                             4277.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.536e-08      0.005   3.09

In [3]:
dat.sample(5)

,firmid,year,lcap,lemp,ldsa
981,82,1977,-0.439564,-0.057154,-0.294509
3123,261,1971,-0.222340,-0.665476,-0.734462
612,52,1968,0.613186,0.380193,-0.072574
4913,410,1973,-2.010690,-1.645560,-1.657590
5287,441,1975,-1.006080,-0.956662,-0.547157


In [4]:
dat.year.unique()

array([1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979])

# Descriptives

In [6]:
dat.describe()

,firmid,lcap,lempl,ldsa
count,5292.000000,5.292000e+03,5.292000e+03,5.292000e+03
mean,221.000000,-7.125472e-09,-1.252834e-08,4.695767e-09
std,127.317437,1.310973e+00,1.180122e+00,1.232499e+00
min,1.000000,-3.864950e+00,-3.382780e+00,-3.551540e+00
25%,111.000000,-9.083267e-01,-7.855270e-01,-9.279720e-01
50%,221.000000,-1.180615e-01,-1.137295e-01,-1.029710e-01
75%,331.000000,9.063340e-01,7.930060e-01,8.562296e-01
max,441.000000,4.103687e+00,3.371332e+00,3.913391e+00


In [7]:
dat[['lcap','lemp','ldsa']].hist();

KeyError: "['lemp'] not in index"

In [ ]:
sns.scatterplot(x='lemp', y='ldsa', data=dat); 

# Converting data to numpy format 

In [ ]:
dat.ldsa.values.shape

In [ ]:
N = dat.firmid.unique().size
T = dat.year.unique().size
assert dat.shape[0] == N*T, f'Error: data is not a balanced panel'
print(f'Data has N={N} and T={T}')

Extract data from `pandas` to `numpy` arrays. 

In [ ]:
y = dat.ldsa.values.reshape((N*T,1))

ones = np.ones((N*T,1))
l = dat.lemp.values.reshape((N*T,1))
k = dat.lcap.values.reshape((N*T,1))
X = np.hstack([ones, l, k])